In [18]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import matplotlib.pyplot as plt
import pickle
import gzip

In [49]:
from line_profiler import * 

In [58]:

import random

def do_stuff(numbers):
    s = sum(numbers)
    l = [numbers[i]/43 for i in range(len(numbers))]
    m = ['hello'+str(numbers[i]) for i in range(len(numbers))]

numbers = [random.randint(1,100) for i in range(1000)]
do_stuff(numbers)
lp = LineProfiler()
lp.print_stats()

Timer unit: 1e-06 s



In [51]:
from line_profiler import LineProfiler

In [52]:
from line_profiler import LineProfiler
import random

class Solution:
    def do_stuff(self,numbers):
        s = sum(numbers)
        l = [numbers[i]/43 for i in range(len(numbers))]
        m = ['hello'+str(numbers[i]) for i in range(len(numbers))]

numbers = [random.randint(1,100) for i in range(1000)]

s = Solution()
lp = LineProfiler()
lp_wrapper = lp(s.do_stuff)
lp_wrapper(numbers)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.001459 s
File: <ipython-input-52-530b22a44142>
Function: do_stuff at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                               def do_stuff(self,numbers):
     6         1         13.0     13.0      0.9          s = sum(numbers)
     7         1        398.0    398.0     27.3          l = [numbers[i]/43 for i in range(len(numbers))]
     8         1       1048.0   1048.0     71.8          m = ['hello'+str(numbers[i]) for i in range(len(numbers))]



In [53]:
class ClusterBlock(nn.Module):
    def __init__(self, n_input, n_output, v ,T, k_cluster):
        super(ClusterBlock, self).__init__( )
        self.n_input = n_input
        self.n_output = n_output
        self.v = v  # number of node
        self.T = T
        self.k_cluster = k_cluster
        self.linear1 = nn.Linear(n_input , 1)  # Dense
        self.linear2 = nn.Linear(T ,k_cluster ) # clustering
        self.softmax = nn.Softmax(dim =-1)
        # support = 3 , each support has k GAT
        self.attentions1 = [GraphAttentionLayer(T, n_input, n_output) for _ in range(k_cluster)]
        self.attentions2 = [GraphAttentionLayer(T, n_input, n_output) for _ in range(k_cluster)]
        self.attentions3 = [GraphAttentionLayer(T, n_input, n_output) for _ in range(k_cluster)]
        self.dropout = nn.Dropout(p=0.5)
    
    
    def forward(self, x, graph_list):  # x.shape = (b,v,T,f)
        b, *_ = x.size()
        # squeeze f
        out = self.linear1(x)    #  out.shape = (b , v ,T,1)
        out = out.view(-1, self.v, self.T)  #  out.shape = (b , v ,T)

        # clustering
        soft_cluster = self.softmax(self.linear2(out))   # out.shape = (b,v,k)
        
        # soft to hard        
#         m = torch.transpose(torch.max(soft_cluster, -1)[0].repeat(2,1),0,1)
        max_mat = torch.max(soft_cluster, -1)[0].unsqueeze(-1).repeat(1, 1, self.k_cluster)
        one_mat = torch.ones(b, self.v, self.k_cluster)
        zero_mat = torch.zeros(b, self.v, self.k_cluster)
        hard_cluster = torch.where(soft_cluster-max_mat>=0, one_mat, zero_mat)
        
        #graph attention
        gout = 0
        for graph in graph_list:   # graph_list : (3,3 ,n , n )
            graph1, graph2, graph3 = graph[0], graph[1], graph[2]
#             cluster_mask_out = torch.zeros(3, b, self.v, self.T*self.output)
            for i in range(self.k_cluster):
                out1 = torch.mul(self.attentions1[i](x, graph1), hard_cluster[:, :, i].unsqueeze(-1))
                out2 = torch.mul(self.attentions2[i](x, graph1), hard_cluster[:, :, i].unsqueeze(-1))
                out3 = torch.mul(self.attentions2[i](x, graph1), hard_cluster[:, :, i].unsqueeze(-1))
                out = F.relu((out1 + out2  + out3)/3 )
            gout += out
#                 cluster_mask_out[0] += out1
#                 cluster_mask_out[1] += out2
#                 cluster_mask_out[2] += out3
#             gout.append(torch.sum(cluster_mask_out, dim=0))

        return gout/3 , hard_cluster 

In [55]:
class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903
    """

    def __init__(self, T , in_features, out_features, concat=False):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = nn.Dropout(p=0.5)
        self.in_features = in_features
        self.out_features = out_features
        self.concat = concat
        self.W = nn.Parameter(torch.zeros(size=(T*in_features, T*out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(2*T*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU()
        
    def forward(self, input, adj): # input.shape = ( b,v, t ,f1)
        B , N , T = input.size()[0], input.size()[1], input.size()[2]     
        
        # h = xw , (b,N,t*f1)  * ( t*f1 ,  t*f2) -> (b ,N , t*f2)
        input = input.view(B*N, -1)
        h = torch.matmul(input, self.W).view(B , N , -1)
        
        # h.repeat(1,1,N) : (b, N, N*t*f2)  
        # h.repeat(1,N,1) : (b ,N*N , t*f2)  
        # output : (b , N , N, 2*t*f2)
        a_input = torch.cat([h.repeat(1, 1, N).view(B, N * N, -1), h.repeat(1, N, 1)], dim=1).view(B, N, N, 2*T*self.out_features)
        # e = a_input* a ,  (b,N,N,2*t*f2) * ( 2*t*f2 ,1) -> ( b ,N ,N )
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(-1))
        zero_vec = -9e15*torch.ones_like(e)
        # adj : (n, n) -> ( b , n ,n )
        b_adj = adj.unsqueeze(0).repeat(B, 1, 1)
        
        attention = torch.where(b_adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=-1)
#         attention = self.dropout(attention, self.dropout, training=self.training)
        
        # (b,n,n) * (b,n ,t*f2)  -> (b, n , t*f2)
        h_prime = torch.bmm(attention, h)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

    
    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'


In [43]:
test_data = torch.rand(10 , 20 , 48 ,1)  # b,v,t, f
test_label = torch.rand(10, 20 , 48 ,1)
graph = torch.rand(20,20)
graph_list = torch.rand(3,3,20,20)

In [44]:
T = 48
in_features = 1 
out_features = 1
v =20 
k_cluster = 2 

In [56]:
model_layer = GraphAttentionLayer(T , in_features, out_features)
model_layer(test_data,graph)

tensor([[[ 0.3428, -0.1388,  0.6871,  ..., -0.3074,  0.8060,  1.0066],
         [ 0.3442, -0.1493,  0.6831,  ..., -0.3246,  0.8110,  0.9747],
         [ 0.3481, -0.1800,  0.6712,  ..., -0.3753,  0.8257,  0.8807],
         ...,
         [ 0.3366, -0.1324,  0.6814,  ..., -0.3129,  0.8261,  0.9882],
         [ 0.3366, -0.1324,  0.6814,  ..., -0.3129,  0.8261,  0.9882],
         [ 0.3366, -0.1324,  0.6814,  ..., -0.3129,  0.8261,  0.9882]],

        [[ 0.3266, -0.1457,  0.5593,  ..., -0.2689,  0.7146,  1.2123],
         [ 0.3262, -0.1453,  0.5594,  ..., -0.2694,  0.7149,  1.2123],
         [ 0.3263, -0.1454,  0.5594,  ..., -0.2692,  0.7148,  1.2123],
         ...,
         [ 0.4050, -0.1546,  0.5467,  ..., -0.2371,  0.7269,  1.2381],
         [ 0.4050, -0.1546,  0.5467,  ..., -0.2371,  0.7269,  1.2381],
         [ 0.4050, -0.1546,  0.5467,  ..., -0.2371,  0.7269,  1.2381]],

        [[ 0.5069, -0.1338,  0.5035,  ..., -0.2158,  0.7635,  1.1479],
         [ 0.5050, -0.1323,  0.5043,  ..., -0

In [57]:
from line_profiler import LineProfiler

model = ClusterBlock( 1, 1, v ,T, k_cluster)
# model_layer(test_data,graph)

model.train()
lp = LineProfiler()
lp_wrapper = lp(model.forward)
y = lp_wrapper(test_data,graph)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.255357 s
File: <ipython-input-53-f187f0627aa8>
Function: forward at line 19

Line #      Hits         Time  Per Hit   % Time  Line Contents
    19                                               def forward(self, x, graph_list):  # x.shape = (b,v,T,f)
    20         1         10.0     10.0      0.0          b, *_ = x.size()
    21                                                   # squeeze f
    22         1        416.0    416.0      0.2          out = self.linear1(x)    #  out.shape = (b , v ,T,1)
    23         1         40.0     40.0      0.0          out = out.view(-1, self.v, self.T)  #  out.shape = (b , v ,T)
    24                                           
    25                                                   # clustering
    26         1        467.0    467.0      0.2          soft_cluster = self.softmax(self.linear2(out))   # out.shape = (b,v,k)
    27                                                   
    28                               

In [40]:
y

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 

In [14]:
model_layer(test_data,graph)

tensor([[[ 0.8512, -0.2636,  0.3433,  ...,  0.5017, -1.4792,  0.8147],
         [ 0.8689, -0.2645,  0.3484,  ...,  0.5139, -1.4695,  0.8050],
         [ 0.7708, -0.2597,  0.3200,  ...,  0.4466, -1.5234,  0.8584],
         ...,
         [ 0.7595, -0.2751,  0.4654,  ...,  0.4289, -1.5504,  0.9308],
         [ 0.7595, -0.2751,  0.4654,  ...,  0.4289, -1.5504,  0.9308],
         [ 0.7595, -0.2751,  0.4654,  ...,  0.4289, -1.5504,  0.9308]],

        [[ 0.7915, -0.0874,  0.5578,  ...,  0.6531, -1.6323,  0.9227],
         [ 0.7218, -0.1052,  0.5227,  ...,  0.5743, -1.6041,  0.9126],
         [ 0.6658, -0.1195,  0.4944,  ...,  0.5109, -1.5814,  0.9046],
         ...,
         [ 0.6678, -0.0461,  0.5616,  ...,  0.6656, -1.6160,  0.9089],
         [ 0.6678, -0.0461,  0.5616,  ...,  0.6656, -1.6160,  0.9089],
         [ 0.6678, -0.0461,  0.5616,  ...,  0.6656, -1.6160,  0.9089]],

        [[ 0.7772, -0.0105,  0.3533,  ...,  0.5313, -1.4442,  0.7907],
         [ 0.7157, -0.0390,  0.4004,  ...,  0

In [13]:
T  = 48
n_input = 1 
n_output = 1 
v = 20 
k_cluster = 3

In [126]:
model = ClusterBlock(n_input, n_output, v ,T, k_cluster)

In [127]:
lp = LineProfiler()
lp_wrapper = lp(model.forward)
lp_wrapper(test_data,graph_list)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.003513 s
File: <ipython-input-111-73d9242b4732>
Function: forward at line 19

Line #      Hits         Time  Per Hit   % Time  Line Contents



In [107]:
model(test_data, graph_list)[0]

tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0057, 0.0000, 0.7051,  ..., 0.0000, 0.3279, 0.1443],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.7105,  ..., 0.0000, 0.3549, 0.1563],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.2179, 0.0000, 0.6480,  ..., 0.0000, 0.2186, 0.1773],
         ...,
         [0.2328, 0.0000, 0.6930,  ..., 0.0000, 0.2687, 0.1587],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.1547, 0.0000, 0.8107,  ..., 0.0000, 0.3011, 0.0470],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.